Our application allows for patients to input some symptoms into a symptom tracker.  This information is then associated to some possible diagnoses.

Some new technologies we used (we did not cover in our boot camp) are:
1. **SentencePiece** is a supplement to our NLTK.  This supplement is needed to assist in translating medical terms or more complex words.
2. **%%capture** is unique to Google Colab.  This allows for the !pip installs to run without generating all the responses, which clutter up the application.
3. **sqlite** is a lightweight database management system.  Given that we are dealing with large dataset(s) for our model, sqlite allows our application to store and retreive data using SQL (structured query language.)  We are using this for efficiency and speed of use.
4. **Flagging** we added this feature to our gradio interface.  It is used to collect information from users about how the application is working. It is part of improving the model over time.

In [ ]:
# Our pip installs needed to run our application.  Note the %%capture being used is for google colab only.
# Remove if you are going to run this in VSCode.

%%capture

!pip install gradio
!pip intall nltk
!pip install transformers
!pip install torch
!pip install sentencepiece
!pip install tensorflow
!pip install tensorflow_hub
!pip install tensorflow_text
!pip install pandas
!pip install numpy
!pip install matplotlib
!pip install seaborn
!pip install sqlite3
!pip install extract_named_entities_nltk




In [ ]:
# Imports needed for this application
import gradio as gr
import torch
import sentencepiece
import tensorflow
import tensorflow_hub
import tensorflow_text
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sqlite3




In [ ]:
from google.colab import files
uploaded = files.upload()

In [ ]:
#  Read the .csv using pandas
import pandas as pd

# Load the data
disease_data = pd.read_csv('symbipredict.csv')

# Display the first few rows of the dataset
print(disease_data.head())

In [ ]:
# After loading the data, it is necessary to combine the symptom_columns into a single column
symptom_columns = [col for col in disease_data.columns if col != 'Disease']
disease_data['Processed_Symptoms'] = disease_data[symptom_columns].apply(lambda x: ' '.join(x.astype(str)), axis=1)

In the section below, we import the necessary libraries and dictionaries in order to build our NLTK model.

In [ ]:
# Import necessary libraries starting with nltk
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd
import gradio as gr
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer

nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
nltk.download('averaged_perceptron_tagger_eng')
nltk.download('maxent_ne_chunker')
nltk.download('words')

from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

In the section below, we are defining how we want to use our dataset(s).  We want patients to input their symptoms, then we associate them to key words from our dataset(s).  This is our preprocessing of the model.


In [ ]:
# Define the prepocessing of the data
def preprocess_symptoms(symptom_text):
    tokens = word_tokenize(symptom_text.lower())
    stop_words = set(stopwords.words('english'))
    filtered_tokens = [w for w in tokens if not w in stop_words and w.isalnum()]
    return ' '.join(filtered_tokens)

# Example synonym dictionary (this can be expanded)
synonym_dict = {
    'fever': ['fever', 'pyrexia'],
    'headache': ['headache', 'migraine', 'cephalalgia'],
    'nausea': ['nausea', 'queasiness', 'sickness'],
    'vomiting': ['vomiting', 'throwing up', 'emesis'],
    'sore throat': ['sore throat', 'pharyngitis', 'throat pain']
}

def expand_keywords(keywords):
    expanded_keywords = set()
    for keyword in keywords:
        if keyword in synonym_dict:
            expanded_keywords.update(synonym_dict[keyword])
        else:
            expanded_keywords.add(keyword)
    return list(expanded_keywords)

def extract_keywords(patient_feedback):
    tokens = word_tokenize(patient_feedback.lower())
    stop_words = set(stopwords.words('english'))
    filtered_tokens = [w for w in tokens if not w in stop_words and w.isalnum()]
    pos_tags = nltk.pos_tag(filtered_tokens)
    keywords = [word for word, pos in pos_tags if pos.startswith('NN') or pos.startswith('JJ') or pos.startswith('VB')]
    return keywords






In the section below, the application will read the patient input and suggest diagnosis.  This is our vectorizing process.  Once we vectorize, we run the gradio app, which generates an input cell for patient data, and an output cell for possible diagnoses.  We chose to use Transformers (vectorizing) via TF-IDF becausee it performed better with gradio.  SpaCy and gradio had constant version conflicts, which caused our application to break, so we switched to TF-IDF.

NOTE:  We are allowing for a possible 5 diagnoses.  Many symptoms cross over numerous diagnoses.  For now, our app is merely suggesting some possible diagnoses.  Our future model will be more precise.  More data is needed to establish that kind of precision.  Given these challenges and the short runway of time we had to develop this application, we decided to put in a patient feedback loop in our gradio application called flagging.  This allows the patient to tell us if the proposed diagnoses are "Correct", "Incorrect", "Needs Improvement".

In [ ]:
import os
import gradio as gr
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Use the preloaded disease_data
def extract_keywords(feedback):
    # Placeholder for actual keyword extraction logic
    return feedback.split()

def expand_keywords(keywords):
    # Placeholder for actual keyword expansion logic
    return keywords

def suggest_diagnosis_tfidf(patient_feedback):
    keywords = extract_keywords(patient_feedback)
    expanded_keywords = expand_keywords(keywords)
    print("Expanded Keywords:", expanded_keywords)

    processed_feedback = ' '.join(expanded_keywords)
    vectorizer = TfidfVectorizer()
    symptom_matrix = vectorizer.fit_transform(disease_data['Processed_Symptoms'])
    feedback_vector = vectorizer.transform([processed_feedback])
    similarities = cosine_similarity(feedback_vector, symptom_matrix)
    sorted_indices = similarities.argsort()[0][::-1]

    possible_diagnoses = []
    added_diseases = set()  # To track added diagnoses and avoid duplicates
    for idx in sorted_indices:
        disease_name = disease_data.loc[idx, 'Disease']
        if disease_name not in added_diseases:
            possible_diagnoses.append(disease_name)
            added_diseases.add(disease_name)

    if not possible_diagnoses:
        possible_diagnoses = ["Unable to determine a diagnosis based on the provided information."]

    return possible_diagnoses[:5]  # Return top 5 possible diagnoses

# Ensure the custom flagged directory exists
custom_flagged_dir = 'custom_flagged_data'
if not os.path.exists(custom_flagged_dir):
    os.makedirs(custom_flagged_dir)
print(f"Flagged data should be saved in: {os.path.abspath(custom_flagged_dir)}")

# Gradio Interface
iface = gr.Interface(
    fn=suggest_diagnosis_tfidf,
    inputs=gr.Textbox(lines=5, placeholder="Describe your symptoms..."),
    outputs="text",
    title="Symptom Checker",
    description="Enter your symptoms, and we'll suggest possible diagnoses.",
    flagging_mode="manual",
    flagging_options=["Correct", "Incorrect", "Needs Improvement"],
    flagging_dir=custom_flagged_dir
)

iface.launch(share=True)



**For Future Development**
 Our future development will consist of expanding our application to include treatment recommendations for the diagnoses output.

